In [10]:
import sys
import subprocess
subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'sklearn'])
subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'pandas'])
subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'numpy'])
from sklearn import metrics
import pandas as pd
import numpy as np
import random
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_curve, auc
from sklearn.svm import LinearSVC
from sklearn.cluster import KMeans
from sklearn import metrics
from copy import deepcopy
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import GridSearchCV,train_test_split

In [11]:
dataset = pd.read_csv("wdbc_without_names.csv",header=None)
cols=["ID","Diagnosis","Attr_1","Attr_2","Attr_3","Attr_4","Attr_5","Attr_6","Attr_7","Attr_8","Attr_9","Attr_10","Attr_11","Attr_12","Attr_13","Attr_14","Attr_15","Attr_16","Attr_17","Attr_18","Attr_19","Attr_20","Attr_21","Attr_22","Attr_23","Attr_24","Attr_25","Attr_26","Attr_27","Attr_28","Attr_29","Attr_30"]
dataset.columns = cols
dataset=dataset.drop("ID",axis=1)
dataset.head()

,Diagnosis,Attr_1,Attr_2,Attr_3,Attr_4,Attr_5,Attr_6,Attr_7,Attr_8,Attr_9,...,Attr_21,Attr_22,Attr_23,Attr_24,Attr_25,Attr_26,Attr_27,Attr_28,Attr_29,Attr_30
0,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [12]:
X = pd.DataFrame()
X = dataset.drop(["Diagnosis"],axis=1)

# Taking Benign =1 Malignant=0

Y=np.where(dataset["Diagnosis"]=="M",0,1)
Y=pd.DataFrame(Y)
Y.columns=["Diagnosis"]
scaler = MinMaxScaler()
scaler.fit(X)
X=scaler.transform(X)
X=pd.DataFrame(X)
normalized_data=pd.concat([X,Y],axis=1)
normalized_data.head()

,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,Diagnosis
0,0.521037,0.022658,0.545989,0.363733,0.593753,0.792037,0.703140,0.731113,0.686364,0.605518,...,0.141525,0.668310,0.450698,0.601136,0.619292,0.568610,0.912027,0.598462,0.418864,0
1,0.643144,0.272574,0.615783,0.501591,0.289880,0.181768,0.203608,0.348757,0.379798,0.141323,...,0.303571,0.539818,0.435214,0.347553,0.154563,0.192971,0.639175,0.233590,0.222878,0
2,0.601496,0.390260,0.595743,0.449417,0.514309,0.431017,0.462512,0.635686,0.509596,0.211247,...,0.360075,0.508442,0.374508,0.483590,0.385375,0.359744,0.835052,0.403706,0.213433,0
3,0.210090,0.360839,0.233501,0.102906,0.811321,0.811361,0.565604,0.522863,0.776263,1.000000,...,0.385928,0.241347,0.094008,0.915472,0.814012,0.548642,0.884880,1.000000,0.773711,0
4,0.629893,0.156578,0.630986,0.489290,0.430351,0.347893,0.463918,0.518390,0.378283,0.186816,...,0.123934,0.506948,0.341575,0.437364,0.172415,0.319489,0.558419,0.157500,0.142595,0


In [13]:
# Taking Positive class=Benign and Negative class=Malignant

X_positive = normalized_data[normalized_data["Diagnosis"]==1]
X_negative = normalized_data[normalized_data["Diagnosis"]==0]

In [14]:
final_unsupervised_train_results = {p:None for p in ['accuracy','precision','recall','fscore','auc']}
final_unsupervised_test_results = {p:None for p in ['accuracy','precision','recall','fscore','auc']}


unsupr_train_results={p:[] for p in ['accuracy','precision','recall','fscore','auc']}

unsupr_test_results={p:[] for p in ['accuracy','precision','recall','fscore','auc']}

for i in range(1,31):
    
    # Randomly selecting Labelled Data (with 50% positive and 50% negative samples) in train set
    
    X_test = pd.concat([X_positive.sample(frac=0.2),X_negative.sample(frac=0.2)])
    X_train=normalized_data.drop(index=X_test.index.tolist())

    y_test=X_test["Diagnosis"]
    y_train=X_train["Diagnosis"]

    X_test = X_test.drop(["Diagnosis"],axis=1)
    X_train = X_train.drop(["Diagnosis"],axis=1)

    X_label, X_unlabel, y_label, y_unlabel = train_test_split(X_train, y_train, test_size=0.50, stratify=y_train)

    X_train=X_train.reset_index(drop=True)
    y_train=y_train.reset_index(drop=True)
    X_test=X_test.reset_index(drop=True)
    y_test=y_test.reset_index(drop=True)
    
    # Computing the centers of two clusters 
    k_means = KMeans(n_clusters=2,init='k-means++',random_state=random.randint(20,200),n_init=20).fit(X_train)
    clus_dist= k_means.transform(X_train)
    #print(k_means.cluster_centers_)

    # Finding the closest 30 data points to each center
    nn = NearestNeighbors(n_neighbors=30,algorithm='brute').fit(X_train)
    distances, indices = nn.kneighbors(k_means.cluster_centers_)

    # Reading the true lables of the 30 datapoints
    clus_0=y_train.loc[indices[0]]
    clus_1=y_train.loc[indices[1]]

    # Taking a majority poll with these 30 points which becomes the label
    # predicted by k-means for the members of each cluster. 
    maj_poll_clus_0=clus_0.value_counts()
    maj_poll_clus_1=clus_1.value_counts()

    # Finding the labels provided by K_means
    pred_labels= k_means.labels_
    pred_labels= pd.DataFrame(pred_labels)

    pred_labels_0 = deepcopy(pred_labels.loc[pred_labels[0] == 0, :])
    pred_labels_1 = deepcopy(pred_labels.loc[pred_labels[0] == 1, :])

    # Compare the labels provided by K-means with the true labels of the training data

    max_index_0=np.argmax(maj_poll_clus_0)
    max_index_1=np.argmax(maj_poll_clus_1)
    pred_labels_0['class']=max_index_0
    pred_labels_1['class']=max_index_1
    final_pred_y=pd.concat([pred_labels_0['class'],pred_labels_1['class']],axis=0)
    final_pred_y=final_pred_y.sort_index()

    # Testing the final Model on Test data
    
    unsupervised_test_results = pd.DataFrame()
    unsupervised_test_results["True_y"] = y_test
    unsupervised_test_results["Pred_y"] = k_means.predict(X_test)


    # Accuracy, Precision, Recall, F-score, and AUC of TRAIN SET and TEST SET

    # Accuracy for Train and Test
    unsupr_train_results['accuracy'].append(accuracy_score(y_train,final_pred_y))
    unsupr_test_results['accuracy'].append(accuracy_score(unsupervised_test_results["True_y"],unsupervised_test_results["Pred_y"]))

    # Precision for Train and Test
    unsupr_train_results['precision'].append(precision_score(y_train,final_pred_y))
    unsupr_test_results['precision'].append(precision_score(unsupervised_test_results["True_y"],unsupervised_test_results["Pred_y"]))

    # Recall for Train and Test
    unsupr_train_results['recall'].append(recall_score(y_train,final_pred_y))
    unsupr_test_results['recall'].append(recall_score(unsupervised_test_results["True_y"],unsupervised_test_results["Pred_y"]))

    # F-score for Train and Test
    unsupr_train_results['fscore'].append(f1_score(y_train,final_pred_y))
    unsupr_test_results['fscore'].append(f1_score(unsupervised_test_results["True_y"],unsupervised_test_results["Pred_y"]))

    # AUC for Train and Test
    fpr, tpr, _ = roc_curve(y_train,final_pred_y)
    area_uc = auc(fpr, tpr)
    unsupr_train_results['auc'].append(auc(fpr, tpr))

    fpr, tpr, _ = roc_curve(unsupervised_test_results["True_y"],unsupervised_test_results["Pred_y"])
    area_uc = auc(fpr, tpr)
    unsupr_test_results['auc'].append(auc(fpr, tpr))

    # Average scores (accuracy, precision, recall, F-score, and AUC) for Train set
    final_unsupervised_train_results['accuracy']=np.mean(unsupr_train_results['accuracy'])
    final_unsupervised_train_results['precision']=np.mean(unsupr_train_results['precision'])
    final_unsupervised_train_results['recall']=np.mean(unsupr_train_results['recall'])
    final_unsupervised_train_results['fscore']=np.mean(unsupr_train_results['fscore'])
    final_unsupervised_train_results['auc']=np.mean(unsupr_train_results['auc'])
    
     # Average scores (accuracy, precision, recall, F-score, and AUC) for Test set
    final_unsupervised_test_results['accuracy']=np.mean(unsupr_test_results['accuracy'])
    final_unsupervised_test_results['precision']=np.mean(unsupr_test_results['precision'])
    final_unsupervised_test_results['recall']=np.mean(unsupr_test_results['recall'])
    final_unsupervised_test_results['fscore']=np.mean(unsupr_test_results['fscore'])
    final_unsupervised_test_results['auc']=np.mean(unsupr_test_results['auc'])
    
print("\n Average Score over 30 Runs for Train set:\n")
print(final_unsupervised_train_results)

print("\n Average Score over 30 Runs for Test set:\n")
print(final_unsupervised_test_results)

/usr/local/Cellar/jupyterlab/2.2.9/libexec/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/Cellar/jupyterlab/2.2.9/libexec/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/Cellar/jupyterlab/2.2.9/libexec/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/Cellar/jupyterlab/2.2.9

/usr/local/Cellar/jupyterlab/2.2.9/libexec/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/Cellar/jupyterlab/2.2.9/libexec/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



 Average Score over 30 Runs for Train set:

{'accuracy': 0.37280701754385964, 'precision': 0.0, 'recall': 0.0, 'fscore': 0.0, 'auc': 0.5}

 Average Score over 30 Runs for Test set:

{'accuracy': 0.32802359882005905, 'precision': 0.3106686926140941, 'recall': 0.30751173708920193, 'fscore': 0.3067709752321527, 'auc': 0.3351050748938073}


/usr/local/Cellar/jupyterlab/2.2.9/libexec/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
